In [45]:
import pandas as pd
import altair as alt

In [46]:
df = pd.read_csv('pokemon_cards.csv')
df

,set abbreviation,set name,card name,price
0,SWSH01,Sword & Shield Base Set,Air Balloon,0.21
1,SWSH01,Sword & Shield Base Set,Air Balloon (Secret),11.49
2,SWSH01,Sword & Shield Base Set,Aurora Energy,0.85
3,SWSH01,Sword & Shield Base Set,Baltoy (101),0.03
4,SWSH01,Sword & Shield Base Set,Baltoy (102),0.03
...,...,...,...,...
2613,SWSH12,Silver Tempest,Worker (Full Art),4.98
2614,SWSH12,Silver Tempest,Worker (Secret),5.93
2615,SWSH12,Silver Tempest,Zeraora,0.11
2616,SWSH12,Silver Tempest,Zubat,0.05


In [47]:
df['set name'].unique()

array(['Sword & Shield Base Set', 'Rebel Clash', 'Darkness Ablaze',
       'Vivid Voltage', 'Battle Styles', 'Chilling Reign',
       'Evolving Skies', 'Fusion Strike', 'Brilliant Stars',
       'Astral Radiance', 'Lost Origin', 'Silver Tempest'], dtype=object)

In [79]:
# mask for aggregatin to apply after group by
agg_dict = {'price': ['count', lambda x: (x > 30).sum(), lambda x: (x > 50).sum(), lambda x: (x > 100).sum(), 'mean', 'sum']}
df_binned = df.groupby('set name').agg(agg_dict)

df_binned.columns = ['card count', '>30', '>50', '>100', 'avg price', 'total set price']

df_ranked = df_binned.sort_values(by=['>100', '>50', '>30', 'avg price'], ascending=False)

In [80]:
df_ranked

,card count,>30,>50,>100,avg price,total set price
set name,,,,,,
Evolving Skies,237,15,12,6,10.615696,2515.92
Chilling Reign,233,11,7,2,6.047339,1409.03
Fusion Strike,284,6,3,2,2.853697,810.45
Lost Origin,217,4,2,2,3.735806,810.67
Battle Styles,183,4,2,1,3.051639,558.45
Brilliant Stars,195,3,2,1,3.081795,600.95
Astral Radiance,216,5,1,1,3.496343,755.21
Silver Tempest,215,5,1,1,3.350698,720.40
Vivid Voltage,203,1,1,1,2.036207,413.35


From the analysis we can see that the evolving skies set comes in at number #1 with 6 cards coming in at >100!

In [123]:
chart = alt.Chart(df_ranked.reset_index(), 
                  title=alt.TitleParams(text = "Ranked Sword and Shield Pokemon Sets",
                                        subtitle = 'Dark Green = Best / Light Green = Worst')).mark_bar().encode(
    x=alt.X('total set price:Q', axis=alt.Axis(title='Total Set Price (USD)', labelFontSize=12, format='$,.2f')),
    y=alt.Y('set name:N', sort=alt.EncodingSortField(field='>100', order='descending'), axis=alt.Axis(title='Pokemon Set', labelFontSize=12)),
    color=alt.Color('>100:Q', scale=alt.Scale(scheme='greens'), legend=alt.Legend(title='Ranking Metric', titleFontSize=12, labelFontSize=0))
).properties(
    width=650,
    height=550
).configure_axis(
    labelFontSize=14,
    titleFontSize=14
).configure_title(fontSize=18)

chart

alt.Chart(...)

We can see from the graph that the sets arent necesarily ranked on their total set price as that can be biased based on how many cards are present in the set... which is why we are ranking the sets based on how many cards fall into each category (>100, >50, > 30, and average price for tie-breakers)